In [3]:
import os
import cv2

In [4]:
images = []
labels = []

In [5]:
!pip uninstall -y kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6

Found existing installation: kaggle 1.5.12
Uninstalling kaggle-1.5.12:
  Successfully uninstalled kaggle-1.5.12
     |████████████████████████████████| 1.7 MB 4.1 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
     |████████████████████████████████| 58 kB 2.7 MB/s             
  Preparing metadata (setup.py) ... done
  Created wheel for kaggle: filename=kaggle-1.5.6-py3-none-any.whl size=72857 sha256=c415366c10ee6b039e41d29c8c4bbf110ef86da52f96d1f99929309c6d12c87a
  Stored in directory: /root/.cache/pip/wheels/aa/e7/e7/eb3c3d514c33294d77ddd5a856bdd58dc9c1fabbed59a02a2b
Successfully built kaggle


In [6]:
import os
os.environ['KAGGLE_USERNAME'] = "avnimittal" # username from the json file
os.environ['KAGGLE_KEY'] = "4bc180dbb8b4fc29c7d4d0b5f6b77707" # Provide your key from the json file
!kaggle competitions download -c fake-image-classification-challenge # api copied from kaggle

 97% 587M/608M [00:24<00:01, 18.7MB/s]
100% 608M/608M [00:24<00:00, 26.4MB/s]


In [7]:
from zipfile import ZipFile

file_name = "/content/fake-image-classification-challenge.zip"

with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('done')

done


In [8]:
!pip install tensorflow

In [9]:
import cv2
import numpy as np
import os
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
import keras
from keras.models import Sequential, Model,load_model
from tensorflow.keras.optimizers import SGD
from keras.callbacks import EarlyStopping,ModelCheckpoint
from google.colab.patches import cv2_imshow
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D,MaxPool2D
from keras.preprocessing import image
from keras.initializers import glorot_uniform

In [10]:
# face_cascade = cv2.CascadeClassifier('C:/Users/Murtaza Hasan/Desktop/Artificial Intelligence/haarcascade_frontalface_default.xml')
for i in ['fake', 'real']:
    imageNameList=os.listdir("data/train/" + str(i))
    for fileName in imageNameList:
        Image=cv2.imread("data/train/" + str(i) + "/" + str(fileName))
        try:
#             gray=cv2.cvtColor(Image,cv2.COLOR_BGR2GRAY)
#             faces = face_cascade.detectMultiScale(gray, 1.1, 10)
            Image=cv2.resize(Image,(64,64))
#             Image=Image/255
    #             print(Image)
    #             print("hello")
            images.append(Image)
            labels.append(i)
#             if i=='real':
#                 print('1')
#                 labels.append(1)

#             else:
#                 print('0')
#                 labels.append(0)
                
        except:
            pass
    print("Inside folder ", i)

Inside folder  fake
Inside folder  real


In [11]:
images=np.array(images)
images=images/255
labels=np.array(labels)

In [12]:
labels

array(['fake', 'fake', 'fake', ..., 'real', 'real', 'real'], dtype='<U4')

In [13]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
labels = le.fit_transform(labels)

In [14]:
labels

array([0, 0, 0, ..., 1, 1, 1])

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, BatchNormalization

In [17]:
trainX,validationX,trainY,validationY=train_test_split(images,labels,test_size=0.3,stratify=labels)

In [18]:
from tensorflow.keras.regularizers import l2

In [19]:

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=l2(0.001), input_shape=(64,64, 3)))
model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=l2(0.001)))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=l2(0.001)))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=l2(0.001)))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.3))

model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=l2(0.001)))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=l2(0.001)))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(256, activation='relu', kernel_initializer='he_uniform', kernel_regularizer=l2(0.001)))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 64, 64, 32)        896       
                                                                 
 batch_normalization (BatchN  (None, 64, 64, 32)       128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 32)        9248      
                                                                 
 batch_normalization_1 (Batc  (None, 64, 64, 32)       128       
 hNormalization)                                                 
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 32)       0         
 )                                                               
                                                        

In [20]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [21]:
from keras import callbacks
earlystopping = callbacks.EarlyStopping(monitor ="val_loss", 
                                        mode ="min", patience = 5, 
                                        restore_best_weights = True)

In [22]:
model.fit(trainX, trainY, validation_data=(validationX, validationY), epochs = 70, callbacks =[earlystopping])

Epoch 1/70
394/394 [==============================] - 30s 43ms/step - loss: 2.0776 - accuracy: 0.6017 - val_loss: 1.9726 - val_accuracy: 0.5076
Epoch 2/70
394/394 [==============================] - 16s 40ms/step - loss: 1.6434 - accuracy: 0.6661 - val_loss: 1.4963 - val_accuracy: 0.6522
Epoch 3/70
394/394 [==============================] - 16s 42ms/step - loss: 1.3468 - accuracy: 0.6914 - val_loss: 1.2431 - val_accuracy: 0.6581
Epoch 4/70
394/394 [==============================] - 16s 40ms/step - loss: 1.1260 - accuracy: 0.7182 - val_loss: 1.1422 - val_accuracy: 0.6700
Epoch 5/70
394/394 [==============================] - 16s 40ms/step - loss: 1.0098 - accuracy: 0.7367 - val_loss: 0.9927 - val_accuracy: 0.7294
Epoch 6/70
394/394 [==============================] - 16s 40ms/step - loss: 0.9125 - accuracy: 0.7551 - val_loss: 0.8406 - val_accuracy: 0.7743
Epoch 7/70
394/394 [==============================] - 16s 40ms/step - loss: 0.8156 - accuracy: 0.7694 - val_loss: 0.8053 - val_accuracy:

In [25]:
model.fit(trainX, trainY, validation_data=(validationX, validationY), epochs = 70, callbacks =[earlystopping])

Epoch 1/70
394/394 [==============================] - 16s 41ms/step - loss: 0.5956 - accuracy: 0.9132 - val_loss: 0.7927 - val_accuracy: 0.8502
Epoch 2/70
394/394 [==============================] - 16s 40ms/step - loss: 0.5899 - accuracy: 0.9123 - val_loss: 0.7363 - val_accuracy: 0.8422
Epoch 3/70
394/394 [==============================] - 16s 40ms/step - loss: 0.6403 - accuracy: 0.9084 - val_loss: 3.9914 - val_accuracy: 0.5572
Epoch 4/70
394/394 [==============================] - 16s 41ms/step - loss: 0.5847 - accuracy: 0.9198 - val_loss: 0.6143 - val_accuracy: 0.8981
Epoch 5/70
394/394 [==============================] - 16s 41ms/step - loss: 0.5446 - accuracy: 0.9271 - val_loss: 0.6285 - val_accuracy: 0.8902
Epoch 6/70
394/394 [==============================] - 16s 41ms/step - loss: 0.5460 - accuracy: 0.9229 - val_loss: 0.5519 - val_accuracy: 0.9137
Epoch 7/70
394/394 [==============================] - 16s 41ms/step - loss: 0.5334 - accuracy: 0.9257 - val_loss: 0.5930 - val_accuracy:

In [26]:
model.fit(trainX, trainY, validation_data=(validationX, validationY), epochs = 70, callbacks =[earlystopping])

Epoch 1/70
394/394 [==============================] - 17s 42ms/step - loss: 0.5556 - accuracy: 0.9278 - val_loss: 0.5762 - val_accuracy: 0.9198
Epoch 2/70
394/394 [==============================] - 16s 40ms/step - loss: 0.5350 - accuracy: 0.9319 - val_loss: 0.5709 - val_accuracy: 0.9061
Epoch 3/70
394/394 [==============================] - 16s 40ms/step - loss: 0.5476 - accuracy: 0.9275 - val_loss: 0.6470 - val_accuracy: 0.8844
Epoch 4/70
394/394 [==============================] - 16s 40ms/step - loss: 0.5407 - accuracy: 0.9323 - val_loss: 0.6379 - val_accuracy: 0.8885
Epoch 5/70
394/394 [==============================] - 16s 40ms/step - loss: 0.5222 - accuracy: 0.9371 - val_loss: 0.8168 - val_accuracy: 0.7941
Epoch 6/70
394/394 [==============================] - 16s 40ms/step - loss: 0.5278 - accuracy: 0.9326 - val_loss: 0.7557 - val_accuracy: 0.8317
Epoch 7/70
394/394 [==============================] - 16s 40ms/step - loss: 0.5099 - accuracy: 0.9396 - val_loss: 0.7720 - val_accuracy:

In [27]:
model.fit(trainX, trainY, validation_data=(validationX, validationY), epochs = 70, callbacks =[earlystopping])

Epoch 1/70
394/394 [==============================] - 16s 41ms/step - loss: 0.5266 - accuracy: 0.9316 - val_loss: 0.6790 - val_accuracy: 0.8722
Epoch 2/70
394/394 [==============================] - 16s 41ms/step - loss: 0.5273 - accuracy: 0.9354 - val_loss: 0.5210 - val_accuracy: 0.9217
Epoch 3/70
394/394 [==============================] - 16s 40ms/step - loss: 0.5062 - accuracy: 0.9385 - val_loss: 0.5934 - val_accuracy: 0.8976
Epoch 4/70
394/394 [==============================] - 16s 40ms/step - loss: 0.5157 - accuracy: 0.9373 - val_loss: 0.6650 - val_accuracy: 0.8630
Epoch 5/70
394/394 [==============================] - 16s 40ms/step - loss: 0.5107 - accuracy: 0.9353 - val_loss: 0.5008 - val_accuracy: 0.9348
Epoch 6/70
394/394 [==============================] - 16s 41ms/step - loss: 0.5109 - accuracy: 0.9356 - val_loss: 0.5424 - val_accuracy: 0.9269
Epoch 7/70
394/394 [==============================] - 16s 41ms/step - loss: 0.5107 - accuracy: 0.9403 - val_loss: 0.6092 - val_accuracy:

In [28]:
model.fit(trainX, trainY, validation_data=(validationX, validationY), epochs = 70, callbacks =[earlystopping])

Epoch 1/70
394/394 [==============================] - 16s 41ms/step - loss: 0.4375 - accuracy: 0.9528 - val_loss: 0.4669 - val_accuracy: 0.9306
Epoch 2/70
394/394 [==============================] - 16s 40ms/step - loss: 0.4266 - accuracy: 0.9541 - val_loss: 0.5089 - val_accuracy: 0.9200
Epoch 3/70
394/394 [==============================] - 16s 40ms/step - loss: 0.4380 - accuracy: 0.9488 - val_loss: 0.5594 - val_accuracy: 0.9128
Epoch 4/70
394/394 [==============================] - 16s 40ms/step - loss: 0.4071 - accuracy: 0.9584 - val_loss: 0.6327 - val_accuracy: 0.8630
Epoch 5/70
394/394 [==============================] - 16s 40ms/step - loss: 0.4350 - accuracy: 0.9507 - val_loss: 0.5421 - val_accuracy: 0.9196
Epoch 6/70
394/394 [==============================] - 16s 40ms/step - loss: 0.4239 - accuracy: 0.9514 - val_loss: 0.4399 - val_accuracy: 0.9461
Epoch 7/70
394/394 [==============================] - 16s 40ms/step - loss: 0.4098 - accuracy: 0.9558 - val_loss: 0.5829 - val_accuracy:

In [29]:
model.fit(trainX, trainY, validation_data=(validationX, validationY), epochs = 70, callbacks =[earlystopping])

Epoch 1/70
394/394 [==============================] - 16s 42ms/step - loss: 0.4237 - accuracy: 0.9543 - val_loss: 0.4466 - val_accuracy: 0.9433
Epoch 2/70
394/394 [==============================] - 16s 40ms/step - loss: 0.3851 - accuracy: 0.9641 - val_loss: 0.4194 - val_accuracy: 0.9378
Epoch 3/70
394/394 [==============================] - 16s 40ms/step - loss: 0.3987 - accuracy: 0.9557 - val_loss: 0.4214 - val_accuracy: 0.9470
Epoch 4/70
394/394 [==============================] - 16s 40ms/step - loss: 0.3976 - accuracy: 0.9577 - val_loss: 0.4393 - val_accuracy: 0.9396
Epoch 5/70
394/394 [==============================] - 16s 41ms/step - loss: 0.3967 - accuracy: 0.9562 - val_loss: 0.6269 - val_accuracy: 0.8802
Epoch 6/70
394/394 [==============================] - 16s 40ms/step - loss: 0.3839 - accuracy: 0.9605 - val_loss: 0.3756 - val_accuracy: 0.9596
Epoch 7/70
394/394 [==============================] - 16s 41ms/step - loss: 0.3788 - accuracy: 0.9590 - val_loss: 0.3798 - val_accuracy:

In [30]:
model.fit(trainX, trainY, validation_data=(validationX, validationY), epochs = 70, callbacks =[earlystopping])

Epoch 1/70
394/394 [==============================] - 16s 42ms/step - loss: 0.3749 - accuracy: 0.9595 - val_loss: 0.4462 - val_accuracy: 0.9357
Epoch 2/70
394/394 [==============================] - 16s 40ms/step - loss: 0.4094 - accuracy: 0.9540 - val_loss: 0.5389 - val_accuracy: 0.9139
Epoch 3/70
394/394 [==============================] - 16s 41ms/step - loss: 0.3828 - accuracy: 0.9600 - val_loss: 0.4144 - val_accuracy: 0.9446
Epoch 4/70
394/394 [==============================] - 16s 40ms/step - loss: 0.3770 - accuracy: 0.9599 - val_loss: 0.4085 - val_accuracy: 0.9544
Epoch 5/70
394/394 [==============================] - 16s 40ms/step - loss: 0.3945 - accuracy: 0.9585 - val_loss: 0.3740 - val_accuracy: 0.9596
Epoch 6/70
394/394 [==============================] - 16s 40ms/step - loss: 0.3785 - accuracy: 0.9585 - val_loss: 0.3963 - val_accuracy: 0.9467
Epoch 7/70
394/394 [==============================] - 16s 41ms/step - loss: 0.3682 - accuracy: 0.9595 - val_loss: 0.3806 - val_accuracy:

In [31]:
model.fit(trainX, trainY, validation_data=(validationX, validationY), epochs = 70, callbacks =[earlystopping])

Epoch 1/70
394/394 [==============================] - 16s 41ms/step - loss: 0.3915 - accuracy: 0.9561 - val_loss: 0.4341 - val_accuracy: 0.9328
Epoch 2/70
394/394 [==============================] - 16s 40ms/step - loss: 0.3689 - accuracy: 0.9621 - val_loss: 0.4114 - val_accuracy: 0.9446
Epoch 3/70
394/394 [==============================] - 16s 40ms/step - loss: 0.3851 - accuracy: 0.9565 - val_loss: 0.3722 - val_accuracy: 0.9615
Epoch 4/70
394/394 [==============================] - 16s 40ms/step - loss: 0.3728 - accuracy: 0.9620 - val_loss: 0.8501 - val_accuracy: 0.8237
Epoch 5/70
394/394 [==============================] - 16s 40ms/step - loss: 0.3870 - accuracy: 0.9565 - val_loss: 0.4535 - val_accuracy: 0.9278
Epoch 6/70
394/394 [==============================] - 16s 40ms/step - loss: 0.3595 - accuracy: 0.9635 - val_loss: 0.7006 - val_accuracy: 0.8898
Epoch 7/70
394/394 [==============================] - 16s 40ms/step - loss: 0.3556 - accuracy: 0.9625 - val_loss: 0.6081 - val_accuracy:

In [32]:
model.fit(trainX, trainY, validation_data=(validationX, validationY), epochs = 70, callbacks =[earlystopping])

Epoch 1/70
394/394 [==============================] - 17s 42ms/step - loss: 0.3739 - accuracy: 0.9597 - val_loss: 0.4470 - val_accuracy: 0.9369
Epoch 2/70
394/394 [==============================] - 16s 40ms/step - loss: 0.3581 - accuracy: 0.9658 - val_loss: 0.4530 - val_accuracy: 0.9259
Epoch 3/70
394/394 [==============================] - 16s 40ms/step - loss: 0.3716 - accuracy: 0.9597 - val_loss: 0.3491 - val_accuracy: 0.9648
Epoch 4/70
394/394 [==============================] - 16s 40ms/step - loss: 0.3421 - accuracy: 0.9659 - val_loss: 0.3951 - val_accuracy: 0.9500
Epoch 5/70
394/394 [==============================] - 16s 41ms/step - loss: 0.3640 - accuracy: 0.9590 - val_loss: 0.4069 - val_accuracy: 0.9454
Epoch 6/70
394/394 [==============================] - 16s 40ms/step - loss: 0.3504 - accuracy: 0.9613 - val_loss: 0.3517 - val_accuracy: 0.9593
Epoch 7/70
394/394 [==============================] - 16s 40ms/step - loss: 0.3436 - accuracy: 0.9637 - val_loss: 0.3403 - val_accuracy:

In [33]:
model.fit(trainX, trainY, validation_data=(validationX, validationY), epochs = 70, callbacks =[earlystopping])

Epoch 1/70
394/394 [==============================] - 16s 41ms/step - loss: 0.3393 - accuracy: 0.9636 - val_loss: 0.4660 - val_accuracy: 0.9144
Epoch 2/70
394/394 [==============================] - 16s 40ms/step - loss: 0.3548 - accuracy: 0.9627 - val_loss: 0.6475 - val_accuracy: 0.8581
Epoch 3/70
394/394 [==============================] - 16s 40ms/step - loss: 0.3499 - accuracy: 0.9637 - val_loss: 0.4803 - val_accuracy: 0.9078
Epoch 4/70
394/394 [==============================] - 16s 40ms/step - loss: 0.3470 - accuracy: 0.9640 - val_loss: 0.3746 - val_accuracy: 0.9533
Epoch 5/70
394/394 [==============================] - 16s 40ms/step - loss: 0.3418 - accuracy: 0.9634 - val_loss: 0.4114 - val_accuracy: 0.9354
Epoch 6/70
394/394 [==============================] - 16s 40ms/step - loss: 0.3520 - accuracy: 0.9617 - val_loss: 0.3552 - val_accuracy: 0.9626
Epoch 7/70
394/394 [==============================] - 16s 40ms/step - loss: 0.3484 - accuracy: 0.9616 - val_loss: 0.5845 - val_accuracy:

In [34]:
earlystopping = callbacks.EarlyStopping(monitor ="val_loss", 
                                        mode ="min", patience = 10, 
                                        restore_best_weights = True)

In [35]:
model.fit(trainX, trainY, validation_data=(validationX, validationY), epochs = 70, callbacks =[earlystopping])

Epoch 1/70
394/394 [==============================] - 16s 41ms/step - loss: 0.3362 - accuracy: 0.9661 - val_loss: 0.3561 - val_accuracy: 0.9528
Epoch 2/70
394/394 [==============================] - 16s 40ms/step - loss: 0.3389 - accuracy: 0.9641 - val_loss: 0.3483 - val_accuracy: 0.9561
Epoch 3/70
394/394 [==============================] - 16s 40ms/step - loss: 0.3272 - accuracy: 0.9665 - val_loss: 0.8629 - val_accuracy: 0.8194
Epoch 4/70
394/394 [==============================] - 17s 42ms/step - loss: 0.3291 - accuracy: 0.9667 - val_loss: 0.3449 - val_accuracy: 0.9578
Epoch 5/70
394/394 [==============================] - 16s 40ms/step - loss: 0.3315 - accuracy: 0.9645 - val_loss: 0.5469 - val_accuracy: 0.8757
Epoch 6/70
394/394 [==============================] - 16s 40ms/step - loss: 0.3377 - accuracy: 0.9630 - val_loss: 0.5518 - val_accuracy: 0.8869
Epoch 7/70
394/394 [==============================] - 16s 41ms/step - loss: 0.3338 - accuracy: 0.9660 - val_loss: 0.4099 - val_accuracy:

In [36]:
model.fit(trainX, trainY, validation_data=(validationX, validationY), epochs = 70, callbacks =[earlystopping])

Epoch 1/70
394/394 [==============================] - 16s 41ms/step - loss: 0.3312 - accuracy: 0.9651 - val_loss: 0.4254 - val_accuracy: 0.9274
Epoch 2/70
394/394 [==============================] - 16s 40ms/step - loss: 0.3268 - accuracy: 0.9661 - val_loss: 0.4583 - val_accuracy: 0.9081
Epoch 3/70
394/394 [==============================] - 16s 41ms/step - loss: 0.3419 - accuracy: 0.9617 - val_loss: 0.3848 - val_accuracy: 0.9474
Epoch 4/70
394/394 [==============================] - 16s 40ms/step - loss: 0.3257 - accuracy: 0.9690 - val_loss: 0.5405 - val_accuracy: 0.8748
Epoch 5/70
394/394 [==============================] - 16s 40ms/step - loss: 0.3245 - accuracy: 0.9671 - val_loss: 0.3757 - val_accuracy: 0.9444
Epoch 6/70
394/394 [==============================] - 16s 40ms/step - loss: 0.3279 - accuracy: 0.9662 - val_loss: 0.5249 - val_accuracy: 0.9080
Epoch 7/70
394/394 [==============================] - 16s 40ms/step - loss: 0.3383 - accuracy: 0.9636 - val_loss: 0.4291 - val_accuracy:

In [38]:
model.save('model_new2.h5')